In [1]:
import torch
import torchvision
from torch import nn
from torchvision import transforms

print(f"torch {torch.__version__} | torchvision {torchvision.__version__}")

torch 2.0.1+cu118 | torchvision 0.15.2+cu118


In [2]:
# import model summary from torchinfo
try:
    from torchinfo import summary
except:
    print(f"[INFO] installing torchinfo...")
    !pip install torchinfo
    from torchinfo import summary

In [3]:
# import modules from sources
from sources import utils, datasetup, engine, models

2023-11-06 22:17:19.664972: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-06 22:17:19.708636: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-06 22:17:20.396412: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# device agnostic
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
# downloading data (10 and 20% training data)
source_10 = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"
source_20 = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip"

image_path_10 = datasetup.download_data(source=source_10, destination="image_10")
image_path_20 = datasetup.download_data(source=source_20, destination="image_20")
image_path_10, image_path_20

[INFO] data/image_10 exist.
[INFO] data/image_20 exist.


(PosixPath('data/image_10'), PosixPath('data/image_20'))

In [6]:
# setup train and test directories
train_dir_10 = image_path_10 / "train"
train_dir_20 = image_path_20 / "train"

# use the same test dir for both datasets
test_dir = image_path_10 / "test"

train_dir_10, train_dir_20, test_dir

(PosixPath('data/image_10/train'),
 PosixPath('data/image_20/train'),
 PosixPath('data/image_10/test'))

In [7]:
# training grid
effnet_version_list = [0, 1, 2]
train_dir_list = [train_dir_10, train_dir_20]
epochs_list = [5, 10]

In [8]:
# create training kernel
def training_kernel(effnet_version: str,
                   train_dir: str,
                   epochs:int,
                   batch_size: int,
                   lr: float,
                   device: torch.device):
            
    grid_name_1 = f"effnet{effnet_version}"
    grid_name_2 = str(train_dir).replace("/","_")
    grid_name_3 = f"epochs_{epochs}"

    # create model and model transforms
    model, model_transforms = models.create_effnet(effnet_version=effnet_version, INFO=False)

    # create dataloaders
    train_dataloader, test_dataloader, class_names = datasetup.create_dataloaders(
        train_dir = train_dir,
        test_dir = test_dir,
        train_transforms=model_transforms,
        test_transforms=model_transforms,
        batch_size=batch_size,
        INFO=False
    )
    # create loss function and optimizer
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

    # perform training grid
    results = engine.train_tsb_writer(
        model = model,
        train_dataloader = train_dataloader,
        test_dataloader = test_dataloader,
        loss_fn = loss_fn,
        optimizer = optimizer,
        epochs=epochs,
        device=device,
        writer=utils.create_writer(grid_names=[
            model.name, grid_name_2, grid_name_3
        ])
    )


In [9]:
BATCH_SIZE = 32

# perform training grid
experiment_num = 0

for effnet_version in effnet_version_list:
    for train_dir in train_dir_list:
        for epochs in epochs_list:
            
            experiment_num += 1
            print("\n")
            print(f"experiment number: {experiment_num} {'***'}")
            
            training_kernel(
                effnet_version=effnet_version,
                train_dir=train_dir,
                epochs=epochs,
                batch_size=BATCH_SIZE,
                lr=0.001,
                device=device
            )
            



experiment number: 1 ***
[INFO] creating writer log at: writerLog/EfficientNet_B0/data_image_10_train/epochs_5


  0%|          | 0/5 [00:00<?, ?it/s]

epoch: 0 | train_loss: 1.0652 | train_acc: 0.4219 | test_loss: 0.9040 | test_acc: 0.6723
epoch: 1 | train_loss: 0.8945 | train_acc: 0.7070 | test_loss: 0.7969 | test_acc: 0.7737
epoch: 2 | train_loss: 0.7273 | train_acc: 0.8555 | test_loss: 0.6653 | test_acc: 0.8352
epoch: 3 | train_loss: 0.7170 | train_acc: 0.7109 | test_loss: 0.5865 | test_acc: 0.8248
epoch: 4 | train_loss: 0.6279 | train_acc: 0.7734 | test_loss: 0.5958 | test_acc: 0.8153


experiment number: 2 ***
[INFO] creating writer log at: writerLog/EfficientNet_B0/data_image_10_train/epochs_10


  0%|          | 0/10 [00:00<?, ?it/s]

epoch: 0 | train_loss: 1.0277 | train_acc: 0.5508 | test_loss: 0.9118 | test_acc: 0.6307
epoch: 1 | train_loss: 0.9067 | train_acc: 0.6328 | test_loss: 0.8153 | test_acc: 0.6212
epoch: 2 | train_loss: 0.7344 | train_acc: 0.7578 | test_loss: 0.7205 | test_acc: 0.7945
epoch: 3 | train_loss: 0.7003 | train_acc: 0.7734 | test_loss: 0.6823 | test_acc: 0.7633
epoch: 4 | train_loss: 0.6010 | train_acc: 0.8945 | test_loss: 0.6035 | test_acc: 0.8144
epoch: 5 | train_loss: 0.5810 | train_acc: 0.7852 | test_loss: 0.5513 | test_acc: 0.8759
epoch: 6 | train_loss: 0.4920 | train_acc: 0.8906 | test_loss: 0.5522 | test_acc: 0.8456
epoch: 7 | train_loss: 0.4733 | train_acc: 0.9023 | test_loss: 0.6029 | test_acc: 0.7746
epoch: 8 | train_loss: 0.4526 | train_acc: 0.9102 | test_loss: 0.5339 | test_acc: 0.8456
epoch: 9 | train_loss: 0.5099 | train_acc: 0.8047 | test_loss: 0.5249 | test_acc: 0.8248


experiment number: 3 ***
[INFO] creating writer log at: writerLog/EfficientNet_B0/data_image_20_train/epochs

...

epoch: 0 | train_loss: 6.4969 | train_acc: 0.1289 | test_loss: 6.3355 | test_acc: 0.0000
epoch: 1 | train_loss: 4.1693 | train_acc: 0.1133 | test_loss: 4.2020 | test_acc: 0.0938
epoch: 2 | train_loss: 2.7785 | train_acc: 0.2500 | test_loss: 3.0064 | test_acc: 0.2377
epoch: 3 | train_loss: 2.1263 | train_acc: 0.3047 | test_loss: 2.3867 | test_acc: 0.2898
epoch: 4 | train_loss: 1.7822 | train_acc: 0.3750 | test_loss: 2.1824 | test_acc: 0.3210
epoch: 5 | train_loss: 1.4473 | train_acc: 0.5664 | test_loss: 1.9940 | test_acc: 0.3826
epoch: 6 | train_loss: 1.3302 | train_acc: 0.4922 | test_loss: 1.9941 | test_acc: 0.4242
epoch: 7 | train_loss: 1.2020 | train_acc: 0.5430 | test_loss: 1.8360 | test_acc: 0.4858
epoch: 8 | train_loss: 1.2144 | train_acc: 0.5859 | test_loss: 1.5614 | test_acc: 0.5890
epoch: 9 | train_loss: 1.0420 | train_acc: 0.6094 | test_loss: 1.5665 | test_acc: 0.6307


experiment number: 11 ***
[INFO] creating writer log at: writerLog/EfficientNet_B2/data_image_20_train/epoch

  0%|          | 0/5 [00:00<?, ?it/s]

epoch: 0 | train_loss: 5.5910 | train_acc: 0.0563 | test_loss: 4.3012 | test_acc: 0.0824
epoch: 1 | train_loss: 2.6481 | train_acc: 0.2979 | test_loss: 2.3554 | test_acc: 0.3409
epoch: 2 | train_loss: 1.7514 | train_acc: 0.4437 | test_loss: 1.7989 | test_acc: 0.4441
epoch: 3 | train_loss: 1.3453 | train_acc: 0.5583 | test_loss: 1.6081 | test_acc: 0.5483
epoch: 4 | train_loss: 1.1137 | train_acc: 0.6396 | test_loss: 1.3586 | test_acc: 0.5900


experiment number: 12 ***
[INFO] creating writer log at: writerLog/EfficientNet_B2/data_image_20_train/epochs_10


  0%|          | 0/10 [00:00<?, ?it/s]

epoch: 0 | train_loss: 5.5289 | train_acc: 0.0896 | test_loss: 4.3073 | test_acc: 0.0625
epoch: 1 | train_loss: 2.6145 | train_acc: 0.3167 | test_loss: 2.4199 | test_acc: 0.2888
epoch: 2 | train_loss: 1.8240 | train_acc: 0.4375 | test_loss: 1.9294 | test_acc: 0.3409
epoch: 3 | train_loss: 1.3894 | train_acc: 0.5563 | test_loss: 1.5746 | test_acc: 0.4858
epoch: 4 | train_loss: 1.1546 | train_acc: 0.6167 | test_loss: 1.3028 | test_acc: 0.6402
epoch: 5 | train_loss: 0.9367 | train_acc: 0.7146 | test_loss: 1.1737 | test_acc: 0.7225
epoch: 6 | train_loss: 0.8994 | train_acc: 0.7250 | test_loss: 1.1296 | test_acc: 0.7330
epoch: 7 | train_loss: 0.7088 | train_acc: 0.8125 | test_loss: 1.0665 | test_acc: 0.7737
epoch: 8 | train_loss: 0.6508 | train_acc: 0.8521 | test_loss: 0.8628 | test_acc: 0.8258
epoch: 9 | train_loss: 0.6784 | train_acc: 0.8063 | test_loss: 0.9690 | test_acc: 0.7945


In [ ]:
%load_ext tensorboard
%tensorboard --logdir writerLog/ --port 0
utils.tb_address()
